In [25]:
import PyPDF2
from openpyxl import Workbook

In [26]:
Pdf1 = 'Datosaprendices.pdf'
Pdf2 = 'Remates.pdf'

Clave_inicio_1 = "Nombre"
Clave_fin_1 = "Apoyo económico"
Clave_vacio = "Nombre"

Clave_2 = ["CÓDIGO", "Fecha del remate", "Ciudad", "Departamento", "Tipo de bien", "Avaluo remate", "Oferta Mínima", "Refencia"] 
Columnas_2 = 8

wb = Workbook()
hoja1 = wb.active
hoja1.title = 'PDF 1'

In [27]:
# Proceso para el primer PDF
with open(Pdf1, 'rb') as archivo_pdf1:
    lector_pdf1 = PyPDF2.PdfReader(archivo_pdf1)
    datos_1 = ""
    capturando_tabla = False
    fila_excel = 1
    for pagina in range(len(lector_pdf1.pages)):
        datos_1 += lector_pdf1.pages[pagina].extract_text()
    filas1 = datos_1.split('\n')
    for fila in filas1:
        if Clave_inicio_1 in fila:
            capturando_tabla = True
        if capturando_tabla:
            if Clave_vacio in fila:
                fila_excel += 1
                fila_excel += 1            
            columnas = fila.split()
            for col_idx, valor in enumerate(columnas, start=1):
                hoja1.cell(row=fila_excel, column=col_idx, value=valor)
            fila_excel += 1
            
        if Clave_fin_1 in fila:
            capturando_tabla = False

In [28]:
# Hoja para el segundo PDF
hoja2 = wb.create_sheet(title='PDF 2')

with open(Pdf2, 'rb') as archivo_pdf2:
    lector_pdf2 = PyPDF2.PdfReader(archivo_pdf2)
    fila_excel = 1
    
    for pagina in range(len(lector_pdf2.pages)):
        texto_completo = lector_pdf2.pages[pagina].extract_text()
        # Separar en líneas según la disposición horizontal
        lineas = texto_completo.split('\n')
        
        # Iterar sobre las líneas para procesar datos
        for linea in lineas:
            # Dividir la línea en partes considerando un mínimo de dos espacios como separación de columnas
            columnas = [parte.strip() for parte in linea.split('  ') if parte.strip()]
            
            if len(columnas) >= Columnas_2 or any(clave in columnas for clave in Clave_2):
                capturando_tabla = True

            if capturando_tabla:
                for col_idx, valor in enumerate(columnas, start=1):
                    hoja2.cell(row=fila_excel, column=col_idx, value=valor)
                fila_excel += 1
                
            if len(columnas) < Columnas_2 and not any(clave in columnas for clave in Clave_2):
                capturando_tabla = False

In [29]:
wb.save('resultado_tablas.xlsx')